In [ ]:
'''
ElasticSearch
vpn連上公司
username: cynthiachan
password: cynthiachan@cynthiachan

brew services start/stop elasticsearch
IP: 192.168.70.136:9200
'''

In [1]:
from elasticsearch import Elasticsearch
import pandas as pd

In [2]:
## 連接主機
es = Elasticsearch(hosts='192.168.70.136', port=9200)

In [3]:
## 查詢此index是否存在
exist = es.indices.exists(index='google-task-log')
print(exist)

True


/opt/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [4]:
#指定要get哪個index的資訊
result = es.indices.get(index='google-task-log') 
print(result)

{'google-task-log': {'aliases': {'.google-task-log': {}}, 'mappings': {'dynamic': 'strict', 'properties': {'query': {'type': 'text', 'norms': False}, 'relatives': {'properties': {'link': {'type': 'keyword', 'ignore_above': 1024}, 'text': {'type': 'text', 'norms': False}}}, 'resultStats': {'type': 'text', 'norms': False}, 'results': {'properties': {'link': {'type': 'keyword', 'ignore_above': 1024}, 'path': {'type': 'text', 'norms': False}, 'snippet': {'type': 'text', 'norms': False}, 'title': {'type': 'text', 'norms': False}}}, 'status': {'type': 'keyword'}, 'taskDuration': {'type': 'float'}, 'taskId': {'type': 'keyword'}, 'trackTime': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss||epoch_millis'}}}, 'settings': {'index': {'codec': 'best_compression', 'number_of_shards': '1', 'routing_partition_size': '1', 'auto_expand_replicas': '0-1', 'provided_name': 'google-task-log', 'default_pipeline': 'track_time', 'creation_date': '1630914092377', 'number_of_replicas': '0', 'uuid': '60izjVznSDy

In [5]:
## 搜尋資料
import json
dsl = {'query': {'match': {'taskId': "1ebd9877f2b06430ad54c7a8da38b31e"}}}
data = es.search(index='google-task-log', body = dsl)
print(json.dumps(data, indent=2, ensure_ascii=False))

{
  "took": 1,
  "timed_out": false,
  "_shards": {
    "total": 1,
    "successful": 1,
    "skipped": 0,
    "failed": 0
  },
  "hits": {
    "total": {
      "value": 1,
      "relation": "eq"
    },
    "max_score": 4.179502,
    "hits": [
      {
        "_index": "google-task-log",
        "_type": "_doc",
        "_id": "1ebd9877f2b06430ad54c7a8da38b31e",
        "_score": 4.179502,
        "_source": {
          "taskDuration": 6.044,
          "query": "振华数据王雪峰情报",
          "relatives": [],
          "resultStats": "約有 16,300 項結果 (搜尋時間：0.29 秒)",
          "trackTime": 1625046154227,
          "results": [
            {
              "snippet": "深圳「振華數據」：如何看待中國國企「監控資料庫」外洩的 ...https://www.bbc.com › trad › chinese-news-54160466https://www.bbc.com › trad › chinese-news-54160466頁庫存檔2020年9月16日 — 「振華數據」的資料庫據報覆蓋約240萬人，引起關注，但有意見指內容大多是公開資料 ... 根據香港媒體報道，公司行政總裁是王雪峰，他曾經在美國國際商業機器（IBM） ... 報道傳出後引起外界關注中國搜集和整理情報的能力。",
              "path": "https://www.bbc.com › trad › chinese-news-54160466",

In [8]:
## 把搜尋資料存到要拿來訓練的csv檔中：每次的搜尋結果都拿來訓練詞向量(剛開始先這樣，因為資料太少，需要慢慢累積)
from csv import writer
length = len(data['hits']['hits'][0]['_source']['results'])
with open('/Users/csti-user/Downloads/OSINT/data/try_1112.xlsx','a+', newline='') as write_obj:
    for i in range(length):
        list_of_elem = [data['hits']['hits'][0]['_source']['results'][i]['link']]
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)

In [9]:
length = len(data['hits']['hits'][0]['_source']['results'])
df = pd.DataFrame(columns=['url'])
for i in range(length):
    df = df.append({'url': data['hits']['hits'][0]['_source']['results'][i]['link']},ignore_index=True)

In [10]:
df.to_excel('/Users/csti-user/Downloads/OSINT/data/try_1112.xlsx',index=False)

In [11]:
df

,url
0,https://www.bbc.com/zhongwen/trad/chinese-news...
1,https://www.epochtimes.com/gb/20/9/15/n1240457...
2,https://www.abc.net.au/chinese/2020-09-14/chin...
3,https://www.huaglad.com/zh-tw/aunews/20200916/...
4,https://newsin.us/%E6%8C%AF%E5%8D%8E%E6%95%B0%...
5,https://www.ntdtv.com/gb/2020/09/14/a102940492...
6,https://twitter.com/watchwci/status/1305282854...
7,https://global.udn.com/global_vision/story/866...
8,https://gnews.org/zh-hant/356177/
9,https://www.enanyang.my/node/352725


In [24]:
dsl = {'query': {'match': {'taskId': "f9fef9bf-bf73-43ab-be19-c91389e4863b"}}}
data = es.search(index='google-task-log', body = dsl, scroll='5m',size=100)
# total = data['hits']['total'] ## 查詢出結果總量
scroll_id = data['_scroll_id']
scroll_id

'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFDFMXzBFM29CZU04bFR5YzFfa2R2AAAAAAARXZUWNlk2MDY0cjJRNGUzX0FrQ3lRdS1uZw=='

In [27]:
data#['hits']['hits'][0]['_source']['results']

{'_scroll_id': 'FGluY2x1ZGVfY29udGV4dF91dWlkDXF1ZXJ5QW5kRmV0Y2gBFDFMXzBFM29CZU04bFR5YzFfa2R2AAAAAAARXZUWNlk2MDY0cjJRNGUzX0FrQ3lRdS1uZw==',
 'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 3.637586,
  'hits': [{'_index': 'google-task-log',
    '_type': '_doc',
    '_id': 'f9fef9bf-bf73-43ab-be19-c91389e4863b',
    '_score': 3.637586,
    '_source': {'taskId': 'f9fef9bf-bf73-43ab-be19-c91389e4863b',
     'taskDuration': 4.049,
     'trackTime': 1610529377663,
     'status': 'finished',
     'query': '振华数据',
     'resultStats': '約有 20,600,000 項結果 (搜尋時間：0.39 秒)',
     'results': [{'link': 'https://zh.wikipedia.org/zh-tw/%E6%B7%B1%E5%9C%B3%E6%8C%AF%E5%8D%8E%E6%95%B0%E6%8D%AE%E4%BF%A1%E6%81%AF%E6%8A%80%E6%9C%AF%E6%9C%89%E9%99%90%E5%85%AC%E5%8F%B8',
       'path': 'zh.wikipedia.org › 深圳振华数据信息技术有限公司',
       'title': '深圳振華數據信息技術有限公司- 維基百科，自由的百科全書',
       'snippet': '深圳振華數據信息

In [25]:
for i in range(0, length+1):
    # scroll參數必须指定否則會報錯
    query_scroll = es.scroll(scroll_id=scroll_id,scroll='5m')['hits']['hits'][0]['_source']['results']
    results += query_scroll

IndexError: list index out of range